In [1]:
import pandas as pd
import numpy as np
import re
import string
from difflib import get_close_matches

In [2]:
houses = pd.read_csv('D:/Data/JacksonvilleHouses.csv', usecols=list(range(1,93)))
print(houses.shape)
houses.head()

C:\Users\robin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(252718, 92)


,RE,section,township,range,tile,mailing_address_1,mailing_address_2,mail_city,mail_state,mail_zipcode,...,unit,city,zipcode,max_price,max_date,most_recent_date,most_recent_price,q_sales_x,q_sales_y,first_seller
0,0000060030R,1,2S,23E,3401.0,2503 SUMMERFIELD LN,NaN,JACKSONVILLE,FL,32234,...,NaN,JACKSONVILLE,32234,192000.0,2004-07-09,2004-07-09,192000.0,1.0,NaN,HIGHSMITH EARL O &LINDA K
1,0000070010R,1,2S,23E,3401.0,C/O TAX DEPARTMENT,9540 SAN JOSE BLVD,JACKSONVILLE,FL,32257,...,NaN,JACKSONVILLE,32234,150000.0,2018-01-04,2018-01-04,150000.0,2.0,1.0,NaN
2,0000090100R,12,2S,23E,3412.0,2495 U S 301 HWY N,NaN,BALDWIN,FL,32234,...,NaN,JACKSONVILLE,32234,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000110000R,12,2S,23E,3412.0,2425 US HWY 301 N,NaN,BALDWIN,FL,32234,...,NaN,JACKSONVILLE,32234,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000120010R,12,2S,23E,3412.0,2204 US HIGHWAY 301 N,NaN,JACKSONVILLE,FL,32234,...,NaN,JACKSONVILLE,32234,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
new_cols = houses.columns.to_list()
new_cols = [x.replace('__','_').replace(' ','_') for x in new_cols]
houses.rename(columns=dict(zip(houses.columns.to_list(),new_cols)), inplace=True)

In [4]:
houses['RE'] = houses['RE'].str.replace('R','')
houses['RE'] = [x[:6] + '-' + x[-4:] for x in houses['RE']]
houses['RE'][:5]

0    000006-0030
1    000007-0010
2    000009-0100
3    000011-0000
4    000012-0010
Name: RE, dtype: object

In [5]:
houses['section'] = houses['section'].astype('category')

In [6]:
houses.drop(columns=['tile'], inplace=True)

In [7]:
care_of = []
for i in range(0,len(houses['mailing_address_1'])):
    if houses['mailing_address_1'][i][:3] == 'C/O':
        co = houses['mailing_address_1'][i].strip().upper()
        co = co.replace('LLC','')
        co = co.replace('INC','')
        co = re.sub("\s+"," ", co)
        co = co.replace('C/O ', '')
        co = co.replace('TAX DEPARTMENT', '')
        CO = co.replace('TAX DPT', '')
        for punctuation in string.punctuation:
            co = co.replace(punctuation, '')
        if co == 'REAL ESTATE DIVISION':
            co = 'CITY OF JACKSONVILLE'
        care_of.append(co.strip())
        houses['mailing_address_1'][i] = str(houses['mailing_address_2'][i])
        houses['mailing_address_2'][i] = ''
    else:
        care_of.append(np.nan)

address_dictionary = \
    {' U S ':' US ', 'P O ':'PO ', 'FLORIDA':'FL', 'STATE ROAD':'SR', 'STATE RD':'SR',
    ' FORT ':' FT ', ' SAINT':' ST', ' TRAIL':' TRL', ' PLACE':' PL', 'CIRCLE':'CIR', ' COURT':' CT', 
    ' BEACH':' BCH',
    ' LANE':' LN', 'STREET':'ST', 'AVENUE':'AVE', ' AV ':' AVE ', 'DRIVE':'DR', ' ROAD':' RD', 
    'HIGHWAY':'HWY',
    'BOULEVARD':'BLVD', ' CREEK':' CK', 'PLAZA':'PLZ',
    'NORTHEAST':'NE', 'NORTHWEST':'NW', 'SOUTHEAST':'SE', 'SOUTHWEST':'SW',
    'NORTH':'N', 'SOUTH':'S', 'EAST':'E', 'WEST':'W', 
    'FIRST':'1ST', 'SECOND':'2ND', 'THIRD':'3RD', 
    'FOURTH':'4TH', 'FIFTH':'5TH', 'SIXTH':'6TH', 'SEVENTH':'7TH', 'EIGHTH':'8TH', 'NINTH':'9TH', 
    'TENTH':'10TH', 'ELEVENTH':'11TH', 'TWELFTH':'12TH', 'THIRTEENTH':'13TH', 'FOURTEENTH':'14TH', 
    'FIFTEENTH':'15TH', 'SIXTEENTH':'16TH', 'SEVENTEENTH':'17TH', 'EIGHTEENTH':'18TH', 'NINETEENTH':'19TH',
    'UNIT':'', 'SUITE':'', ' STE ':' ', ' SET ':'', 'BUILDING':'', 'BLDNG':'', 'APARTMENT':'',
    ' APT ':''}

houses['mailing_address_1'] = [re.sub("\s+"," ", x) for x in houses['mailing_address_1']]
houses['mailing_address_1'] = houses['mailing_address_1'].str.strip().str.upper()
for punctuation in string.punctuation:
    houses['mailing_address_1'] = houses['mailing_address_1'].str.replace(punctuation, '')

for k in list(address_dictionary.keys()):
    houses['mailing_address_1'] = houses['mailing_address_1'].str.replace(k, address_dictionary[k])
    
houses['care_of'] = care_of

<ipython-input-7-29c7522ef217>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['mailing_address_1'][i] = str(houses['mailing_address_2'][i])
<ipython-input-7-29c7522ef217>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['mailing_address_2'][i] = ''
<ipython-input-7-29c7522ef217>:40: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  houses['mailing_address_1'] = houses['mailing_address_1'].str.replace(punctuation, '')


In [8]:
houses['mailing_address_2'] = houses['mailing_address_2'].fillna('').astype('str')

for i in range(0,len(houses['mailing_address_2'])):
    if houses['mailing_address_2'][i][:3] == 'C/O':
        houses['care_of'][i] = houses['mailing_address_2'][i].replace('C/O ','').replace('TAX DEPARTMENT','')
        houses['mailing_address_2'][i] = ''

houses['mailing_address_2'] = [re.sub("\s+"," ", x) for x in houses['mailing_address_2']]
houses['mailing_address_2'] = houses['mailing_address_2'].str.strip().str.upper()
for punctuation in string.punctuation:
    houses['mailing_address_2'] = houses['mailing_address_2'].str.replace(punctuation, '')
for k in list(address_dictionary.keys()):
    houses['mailing_address_2'] = houses['mailing_address_2'].str.replace(k, address_dictionary[k])

houses['mailing_address'] = houses['mailing_address_1'] + ' ' + houses['mailing_address_2'].map(str)
houses['mailing_address'] = houses['mailing_address'].str.strip()

houses.drop(columns=['mailing_address_1', 'mailing_address_2'], inplace=True)

<ipython-input-8-3f8141bee535>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['care_of'][i] = houses['mailing_address_2'][i].replace('C/O ','').replace('TAX DEPARTMENT','')
<ipython-input-8-3f8141bee535>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['mailing_address_2'][i] = ''
<ipython-input-8-3f8141bee535>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  houses['mailing_address_2'] = houses['mailing_address_2'].str.replace(punctuation, '')


In [9]:
houses['mail_city'] = houses['mail_city'].fillna('NONE').astype('str').str.upper().str.strip()
houses['mail_city'] = [re.sub(r'[0-9]+', '', x) for x in houses['mail_city']]
houses['mail_city'] = [re.sub("\s+"," ", x) for x in houses['mail_city']]

for punctuation in string.punctuation:
    houses['mail_city'] = houses['mail_city'].str.replace(punctuation, '')

first_replacements = \
    {' ON ':' ONTARIO ', 'ONTARIO THE':'ON THE', ' GU ':' GUAM ', 'ISREAL':'ISRAEL', ' IL ':' ISRAEL ',
     'HONG KONG':'CHINA', ' CA ':' CANADA ', 'BCH':'BEACH', 'TOKYO':'JAPAN', 'LONDON':'UNITED KINGDOM',
     'TRINIDAD':'TRINIDAD AND TOBEGO', 'ZURICH':'SWITZERLAND', 'ONTARIO':'CANADA', 'TIKVA':'ISRAEL',
     'HAAYIN':'ISRAEL', 'TEL AVIV':'ISRAEL', 'KOREA':'SOUTH KOREA', 'MAIMI':'MIAMI'
    }

for k in list(first_replacements.keys()):
    houses['mail_city'] = houses['mail_city'].str.replace(k, first_replacements[k])

houses['mail_city'] = ['SAINT'+x[3:] if x[:3] == 'ST ' else x for x in houses['mail_city']]
houses['mail_city'] = ['FORT'+x[3:] if x[:] == 'FT' else x for x in houses['mail_city']]
houses['mail_city'] = ['MOUNT'+x[3:] if x[:3] == 'MT ' else x for x in houses['mail_city']]
houses['mail_city'] = [x[2:] if x[:2] in ['E ','W ','N ','S '] else x for x in houses['mail_city']]
houses['mail_city'] = [x[6:] if x[:6] in ['NORTH ','SOUTH '] else x for x in houses['mail_city']]
houses['mail_city'] = [x[5:] if x[:5] in ['EAST ','WEST '] else x for x in houses['mail_city']]
houses['mail_city'] = ['ISRAEL' if x[-3:] == ' IL' else x for x in houses['mail_city']]

for i in range(0, len(houses['mail_city'])):
    if houses['mail_city'][i] == 'NY':
        houses['mail_city'][i] = 'NEW YORK'
    elif houses['mail_city'][i] == 'JAX':
        houses['mail_city'][i] = 'JACKSONVILLE'

country_list = [
    'SPAIN', 'ISRAEL', 'ETHIOPIA', 'CANADA', 'JAPAN', 'LEBANON','AUSTRALIA', 'COLUMBIA', 'BRAZIL', 'CHINA',
    'KOREA', 'TURKEY', 'BAHAMAS', 'VIETNAM', 'PERU', 'NEW ZEALAND', 'FRANCE', 'BERMUDA', 'SWITZERLAND',
    'TRINIDAD AND TOBEGO', 'ECUADOR', 'UNITED KINGDOM', 'CZECH REPUBLIC', 'JAMAICA']

for country in country_list:
    houses['mail_city'] = \
    ['FOREIGN COUNTRY' if x.__contains__(country) == True else x for x in houses['mail_city']]

city_list = [
    'JACKSONVILLE', 'JACKSONVILLE BEACH', 'SAINT AUGUSTINE', 'SAINT AUGUSTINE BEACH', 'ATLANTIC BEACH',
    'ALEXANDRIA', 'ATLANTA', 'AMELIA ISLAND', 'BALDWIN', 'BALDWIN PARK', 'BATON ROUGE', 'BOCA RATON',
    'CINCINNATI', 'PALATKA', 'FERNANDINA', 'FERNANDINA BEACH', 'FLEMING ISLAND', 'FLAGLER BEACH', 
    'FRONT ROYAL', 'GRAND PRAIRIE', 'HIGHSTOWN', 'HOBOKEN', 'JAMAICA', 'JAMAICA ESTATES', 'JAMAICA PLAIN',
    'KEY BISCAYNE', 'LAS VEGAS', 'MANHATTAN', 'MIAMI', 'MIAMI BEACH', 'MIAMI GARDENS', 'MIAMI LAKES',
    'MIAMI SHORES', 'MIAMI SPRINGS', 'MIAMISBURG', 'MARIETTA', 'MIDDLEBURG', 'NEPTUNE BEACH', 
    'NEW ORLEANS', 'NEW SMYRNA', 'NEW SMYRNA BEACH', 'OCALA', 'PONTE VEDRA BEACH', 'PONTE VEDRA',
    'SAINT PETERSBURG', 'SAINT PETERSBURG BEACH', 'SARATOGA', 'TALLAHASSEE', 'WARNER ROBBINS', 'YULEE'
    'INDIANAPOLIS', 'LOS ANGELES', 'FORT LAUDERDALE', 'FORT COLLINS', 'LONG BEACH']

houses['mail_city'] = \
    [get_close_matches(x, city_list, n=1, cutoff=0.8)[0] if len(get_close_matches(x, city_list, \
    n=1, cutoff=0.8)) > 0 else x for x in houses['mail_city']]

<ipython-input-9-12f921321204>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  houses['mail_city'] = houses['mail_city'].str.replace(punctuation, '')
<ipython-input-9-12f921321204>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['mail_city'][i] = 'JACKSONVILLE'
<ipython-input-9-12f921321204>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['mail_city'][i] = 'NEW YORK'


In [10]:
houses['mail_state'] = houses['mail_state'].fillna('FOREIGN COUNTRY').str.replace('FG','GA')

In [11]:
#houses['mail_zipcode'] = houses['mail_zipcode'].astype('str').fillna('FOREIGN COUNTRY')
#houses['mail_zipcode'] = [x[:-2] if x[-2:] == '.0' else x for x in houses['mail_zipcode']]
#houses['mail_zipcode'].value_counts()[:10]
zips = houses['mail_zipcode'].fillna('00000').str.replace(' ','0').str.strip()

In [12]:
houses.rename(columns={'subdvision_name':'subdivision_name'}, inplace=True)
houses['subdivision_name'].value_counts()

SECTION LAND                 8897
JAX HEIGHTS SEC              1535
MURRAY HILLS HEIGHTS         1451
OAKWOOD VILLA ESTATES        1175
SRINGFELD S/D BLK 3,5,9 ,     967
                             ... 
PALERMO PHASE 1-A               1
BRINKLEYS S/D LOT 7 BLK M       1
GOUINS S/D BLK 1                1
FISHERS PLAT                    1
YOUNGS R/P PT BLK 13 LIND       1
Name: subdivision_name, Length: 5167, dtype: int64

In [13]:
houses['subdivision_name'] = houses['subdivision_name'].fillna('SECTION LAND')

In [14]:
houses['neighborhood'] = houses['neighborhood'].astype('str')

In [15]:
houses['just_value'] = houses['just_value'].astype('int64')
houses = houses[houses['just_value'] > 0]

In [16]:
houses['perc_school_deduction'] = houses['perc_school_deduction'].fillna(0)
houses['perc_school_deduction'] = houses['perc_school_deduction']/100

In [17]:
houses['perc_county_deduction'] = houses['perc_county_deduction'].fillna(0)
houses['perc_county_deduction'] = houses['perc_county_deduction']/100

In [18]:
houses['tax_district'] = houses['tax_district'].astype('category')

In [19]:
houses['lot_sf'] = houses['lot_sf'].astype('int64')

In [20]:
houses['type_descr'] = houses['type_descr'].astype('category')

In [21]:
houses.drop(columns=['style'], inplace=True)

In [22]:
houses['class'] = houses['class'].astype('category')

In [23]:
houses['quality'] = houses['quality'].astype('category')

In [24]:
houses['building_value'] = houses['building_value'].astype('int64')

In [25]:
houses['street_num'] = houses['street_num'].fillna(0).astype('int64').astype('str')
houses['direction'] = houses['direction'].fillna('')
houses['street_name'] = houses['street_name'].fillna('NOT LISTED')
houses['street_type'] = houses['street_type'].fillna('')
houses['unit'] = houses['unit'].fillna('0')

site_address = houses['street_num'] + ' ' + houses['direction'].map(str) + ' ' + \
    houses['street_name'].map(str) + ' ' + houses['street_type'].map(str) + ' ' + houses['unit'].map(str)
site_address = [x[:-2] if x[-2:] == ' 0' else x for x in site_address]
site_address = [re.sub("\s+"," ", x) for x in site_address]
houses['site_address'] = site_address
houses.drop(columns=['street_num', 'direction', 'street_name', 'street_type', 'unit'], inplace=True)

In [26]:
houses.tail()

,RE,section,township,range,mail_city,mail_state,mail_zipcode,subdivision_name,neighborhood,perc_capped,...,max_price,max_date,most_recent_date,most_recent_price,q_sales_x,q_sales_y,first_seller,care_of,mailing_address,site_address
252713,181770-0100,38,3S,29E,JACKSONVILLE,FL,32224.0,SECTION LAND,314408.27,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5055 DIXIE LANDING DR,5055 DIXIE LANDING DR
252714,181771-0200,38,3S,29E,JACKSONVILLE,FL,32224.0,SECTION LAND,314408.27,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5147 DIXIE LANDING DR,5147 DIXIE LANDING DR
252715,181773-0000,38,3S,29E,JACKSONVILLE,FL,32224.0,SECTION LAND,314408.27,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5187 DIXIE LANDING DR,5187 DIXIE LANDING DR
252716,181774-0000,38,3S,29E,JACKSONVILLE,FL,32224.0,SECTION LAND,314408.27,1.0,...,759500.0,1998-01-22,1998-01-22,759500.0,1.0,NaN,NaN,NaN,5161 DIXIE LANDING DR,5161 DIXIE LANDING DR
252717,181776-0000,42,3S,29E,JACKSONVILLE,FL,32250.0,SECTION LAND,314408.26,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4351 PORT ARTHUR CT,4351 PORT ARTHUR CT


get_close_matches of addresses like below

In [27]:
investors = pd.DataFrame(houses.care_of.value_counts())
investors.head()

,care_of
INVITATION HOMES,868
TRICON AMERICAN HOMES,309
HAVENBROOK HOMES,134
FIRST KEY HOMES,77
WRI PROPERTY MANAGEMENT,72


In [28]:
investors[investors.care_of >= 5].sort_index()

,care_of
ADAMS ROTHSTEIN SIEGEL P A,5
AMERICAN HOMES 4 RENT PROP TAX DEPT,6
CATHERINE F MOBLEY,5
CITY OF JACKSONVILLE,20
FIRST KEY HOMES,77
GREEN CREEK ASSET MANAGEMENT,13
HAVENBROOK HOMES,134
HORIZON TRUST COMPANY,10
INVITATION HOMES,868
JEANNETTE C HOLMES,7


In [37]:
houses[houses['care_of']=='TRICON AMERICAN HOMES']['mailing_address'].value_counts().index.to_list()[0]

'1508 BROOKHOLLOW DR'

In [33]:
addresses = pd.DataFrame(houses.mailing_address.value_counts())
addresses.head()

,mailing_address
PO BOX 4090,1239
30601 AGOURA RD 200,978
1717 MAIN ST 2000,675
3721 DUPONT STATION CT S,500
1508 BROOKHOLLOW DR,404


In [35]:
addresses[addresses.mailing_address >= 50]

,mailing_address
PO BOX 4090,1239
30601 AGOURA RD 200,978
1717 MAIN ST 2000,675
3721 DUPONT STATION CT S,500
1508 BROOKHOLLOW DR,404
1717 MAIN ST 2000,377
5001 PLZ ON THE LAKE 200,309
3721 DUPONT STATION CT,246
PO BOX 1459,238
3500 S DUPONT HWY,215


In [3]:
houses.columns

Index(['RE', 'section', 'township', 'range', 'tile', 'mailing_address_1',
       'mailing_address_2', 'mail_city', 'mail_state', 'mail_zipcode',
       'subdvision_name', 'neighborhood', 'perc_capped', 'just_value',
       'perc_school_deduction', 'perc_county_deduction', 'tax_district',
       'lot_sf', 'building', 'type_descr', 'style', 'class', 'quality',
       'actual_yr_built', 'age', 'effec_yr_built', 'effec_age',
       'building_value', 'heated_sf', 'ft_depr_val', 'ft__Carport Al',
       'ft__Cov Patio', 'ft__Deck Wd', 'ft__Firep Gas', 'ft__Firep Ms',
       'ft__Firep Prf', 'ft__Other', 'ft__Pool', 'ft__Scr Porch',
       'ft__Screen En', 'ft__Shed wood', 'baths', 'bedrooms', 'stories',
       'rooms', 'd__CANAL', 'd__GOLF COURSE', 'd__LAKE',
       'd__Local Riverside Avondale Historic District',
       'd__Local Springfield Historic District', 'd__MISCELLANEOUS WATERWAY',
       'd__National Register Ortega Historic District', 'd__OCEAN FRONT',
       'd__Other', 'd__ST JO